In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('/Users/etiennecampredon/Projects/green_steel_plant_FR/ML_file_min_max_avg.csv')
df['Date'] = pd.to_datetime(df['Date'])

filter_Date = (df['Date'] > '2017-12-31') & (df['Date'] < '2022-01-01')
filter_Grand_Est = df['Region'] == 'Grand Est'
filter_PACA = df['Region'] == 'Provence-Alpes-Côte d\'Azur'
filter_Auvergne_Rhone_Alpes = df['Region'] == 'Auvergne-Rhône-Alpes'

# Define regional series 
series_GE = df[(filter_Date) & (filter_Grand_Est)]
series_GE = series_GE.set_index('Date')
series_GE = series_GE['Total_cons_MWh']

series_PACA = df[(filter_Date) & (filter_PACA)]
series_PACA = series_PACA.set_index('Date')
series_PACA = series_PACA['Total_cons_MWh']
  
series_ARA = df[(filter_Date) & (filter_Auvergne_Rhone_Alpes)]
series_ARA = series_ARA.set_index('Date')
series_ARA = series_ARA['Total_cons_MWh']

print(series_GE)
print(series_PACA)
print(series_ARA)

# define industry plant series 
df_steel = pd.read_csv('/Users/etiennecampredon/Projects/green_steel_plant_FR/industry_15mn_call_per_day.csv')
#df_steel['date_date'] = df_steel['truncated'].str.rstrip('T00:00:00')

#setting column date_date to datetime and removing the time before grouping by date
df_steel['date_date'] = pd.to_datetime(df_steel['date_date'], yearfirst = True)
df_steel['date_date'] = df_steel['date_date'].dt.date

# grouping by date 
df_steel = df_steel.groupby('date_date').agg({'Usage_kWh' : 'sum'})
#print(df_steel) 

series_steel = df_steel['Usage_kWh']
print(series_steel)


Date
2018-01-01    230837.0
2018-01-02    267965.0
2018-01-03    285797.0
2018-01-04    288602.0
2018-01-05    281801.0
                ...   
2021-12-27    238779.0
2021-12-28    244126.0
2021-12-29    237856.0
2021-12-30    223151.0
2021-12-31    219623.0
Name: Total_cons_MWh, Length: 1461, dtype: float64
Series([], Name: Total_cons_MWh, dtype: float64)
Date
2018-01-01    381289.0
2018-01-02    429409.0
2018-01-03    437054.0
2018-01-04    438148.0
2018-01-05    415179.0
                ...   
2021-12-27    410482.0
2021-12-28    402913.0
2021-12-29    389557.0
2021-12-30    363036.0
2021-12-31    366208.0
Name: Total_cons_MWh, Length: 1461, dtype: float64
date_date
2018-01-01     351.86
2018-01-02    3950.43
2018-01-03    3561.05
2018-01-04    4977.72
2018-01-05    4683.40
               ...   
2018-12-27    2613.87
2018-12-28    1838.58
2018-12-29     377.42
2018-12-30     339.20
2018-12-31     339.08
Name: Usage_kWh, Length: 365, dtype: float64


In [2]:
# standardize data 

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

standardized_series_GE = scaler.fit_transform(np.array(series_GE).reshape(-1, 1))
standardized_series_PACA = scaler.fit_transform(np.array(series_PACA).reshape(-1, 1))
standardized_series_ARA = scaler.fit_transform(np.array(series_ARA).reshape(-1, 1))

standardized_series_steel = scaler.fit_transform(np.array(series_steel).reshape(-1, 1))

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by StandardScaler.

In [ ]:
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

def dtw_distance(series1, series2):
    n = len(series1)
    m = len(series2)
    dtw = np.zeros((n+1, m+1))
    for i in range(n+1):
        dtw[i, 0] = np.inf
    for j in range(m+1):
        dtw[0, j] = np.inf
    dtw[0, 0] = 0

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(series1[i-1] - series2[j-1])
            dtw[i, j] = cost + min(dtw[i-1, j], dtw[i, j-1], dtw[i-1, j-1])

    return dtw[n, m]

# Example usage
distance_ARA = dtw_distance(standardized_series_ARA, standardized_series_steel)
print("DTW distance to ARA:", distance_ARA)

distance_PACA = dtw_distance(standardized_series_PACA, standardized_series_steel)
print("DTW distance to PACA:", distance_PACA)

distance_GE = dtw_distance(standardized_series_GE, standardized_series_steel)
print("DTW distance to GE:", distance_GE)


In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt 
import numpy as np

import pytz
from datetime import datetime

x_ARA = series_ARA.index
y_ARA = series_ARA.rolling(window = 7).sum()

x_PACA = series_PACA.index
y_PACA = series_PACA.rolling(window = 7).sum()

x_GE = series_GE.index
y_GE = series_GE.rolling(window = 7).sum()

x_steel = series_steel.index
y_steel = series_steel.rolling(window = 15).sum()

fig, axs = plt.subplots(4, figsize = (12,18))

# Auvergne Rhone Alpes plot
axs[0].plot(x_ARA, y_ARA, label = 'ARA')
axs[0].set_title('ARA')

axs2 = axs[0].twinx()
axs2.plot(x_ARA, y_steel, label = 'steel', color = 'red', linestyle = 'dotted')

# Occitanie plot
axs[1].plot(x_OCC, y_OCC, label = 'PACA')
axs[1].set_title('PACA')

axs2 = axs[1].twinx()
axs2.plot(x_PACA, y_steel, label = 'steel', color = 'red', linestyle = 'dotted')

axs[2].plot(x_GE, y_GE, label = 'GE')
axs[2].set_title('GE')

axs2 = axs[2].twinx()
axs2.plot(x_GE, y_steel, label = 'steel', color = 'red', linestyle = 'dotted')

axs[3].plot(x_steel, y_steel, color = 'red', linestyle = 'dotted')
axs[3].set_title('Steel plant')


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

series_steel.index = pd.to_datetime(series_steel.index)
series_ARA.index = pd.to_datetime(series_ARA.index)
series_PACA.index = pd.to_datetime(series_PACA.index)
series_GE.index = pd.to_datetime(series_GE.index)

result_steel = seasonal_decompose(x = series_steel, model='multiplicative')
result_steel.plot()
plt.show()

result_ARA = seasonal_decompose(x = series_ARA, model='multiplicative')
result_ARA.plot()
plt.show()

result_PACA = seasonal_decompose(x = series_PACA, model='multiplicative')
result_PACA.plot()
plt.show()

result_GE = seasonal_decompose(x = series_GE, model='multiplicative')
result_GE.plot()
plt.show()

result_steel.seasonal
result_ARA.seasonal
result_PACA.seasonal
result_GE.seasonal

result_steel.trend
result_ARA.trend
result_PACA.trend
result_GE.trend

In [ ]:
series_seasonality_steel = result_steel.seasonal
series_seasonality_ARA = result_ARA.seasonal
series_seasonality_PACA = result_PACA.seasonal
series_seasonality_GE = result_GE.seasonal


distance_seasonality_ARA = dtw_distance(series_seasonality_GE, series_seasonality_steel)
print("DTW distance to ARA:", distance_seasonality_ARA)

distance_seasonality_PACA = dtw_distance(series_seasonality_PACA, series_seasonality_steel)
print("DTW distance to PACA:", distance_seasonality_PACA)

distance_seasonality_GE = dtw_distance(series_seasonality_ARA, series_seasonality_steel)
print("DTW distance to GE:", distance_seasonality_GE)